<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/assignments/assignment_yourname_class6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/index.html)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Module 6 Assignment: Extract Text with LLM**

**Student Name: Your Name**

# Assignment Instructions

A [file](https://s3.amazonaws.com/data.heatonresearch.com/data/t81-558/sentences.csv) is provided that contains 100 English sentences. Sample sentences from this file include:

|id|sentence|
|---|---|
|1|Sarah found an old photograph in the attic.|
|2|By the window, Jake noticed a sparkling diamond necklace.|
|3|The antique clock was expertly fixed by Robert.|
|4|At the beach, Maria stumbled upon a washed-up bottle.|
|...|...|

For each of these sentences you should extract the name of the person from the sentence. The results of this assignment would look like the following for the above input.

|id|name|
|---|---|
|1|Sarah|
|2|Jake|
|3|Robert|
|4|Maria|
|...|...|

Use a large language model (LLM) to extract the single word action from each of these sentences.



# Google CoLab Instructions

If you are using Google CoLab, it will be necessary to mount your GDrive so that you can send your notebook during the submit process. Running the following code will map your GDrive to ```/content/drive```.

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


# LangChain Setup

We must first install LangChain, refer to Module 6.2 for more information.

In [ ]:
!pip install langchain langchain_openai

You will need a key for this assignment, for WUSTL students, look at Assignment 6 in Canvas.

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI

# Your OpenAI API key
# If you are in my class at WUSTL, get this key from the Assignment 6 description in Canvas.
OPENAI_KEY = ''

# This is the model you will generally use for this class
LLM_MODEL = 'gpt-3.5-turbo-instruct'

# Initialize the OpenAI LLM (Language Learning Model) with your API key
llm = OpenAI(openai_api_key=OPENAI_KEY, model=LLM_MODEL, temperature=0)

# Assignment Submit Function

You will submit the 10 programming assignments electronically.  The following submit function can be used to do this.  My server will perform a basic check of each assignment and let you know if it sees any basic problems.

**It is unlikely that should need to modify this function.**

In [ ]:
import base64
import os
import numpy as np
import pandas as pd
import requests
import PIL
import PIL.Image
import io

# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - List of pandas dataframes or images.
# key - Your student key that was emailed to you.
# course - The course that you are in, currently t81-558 or t81-559.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,course,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    payload = []
    for item in data:
        if type(item) is PIL.Image.Image:
            buffered = BytesIO()
            item.save(buffered, format="PNG")
            payload.append({'PNG':base64.b64encode(buffered.getvalue()).decode('ascii')})
        elif type(item) is pd.core.frame.DataFrame:
            payload.append({'CSV':base64.b64encode(item.to_csv(index=False).encode('ascii')).decode("ascii")})
    r= requests.post("https://api.heatonresearch.com/wu/submit",
        headers={'x-api-key':key}, json={ 'payload': payload,'assignment': no, 'course':course, 'ext':ext, 'py':encoded_python})
    if r.status_code==200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Assignment #6 Sample Code

The following code provides a starting point for this assignment.

In [ ]:
import os
import pandas as pd
from scipy.stats import zscore
import string
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# This is your student key that I emailed to you at the beginnning of the semester.
# key = "uTtH5yNbPs9tZdRWsBf9V9FaQA9RU2iP5cL7F3zH" #"Gx5en9cEVvaZnjut6vfLm1HG4ZO4PsI32sgldAXj"  # This is an example key and will not work.

key = ""

# You must also identify your source file.  (modify for your local setup)
# file='/content/drive/My Drive/Colab Notebooks/assignment_yourname_class6.ipynb'  # Google CoLab
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\assignments\\assignment_yourname_class6.ipynb'  # Windows
# file='/Users/jheaton/projects/t81_558_deep_learning/assignments/assignment_yourname_class6.ipynb'  # Mac/Linux

file= '/content/drive/My Drive/Colab Notebooks/assignment_ZhijiangLi_class6.ipynb'

# Begin assignment

df = pd.read_csv("https://s3.amazonaws.com/data.heatonresearch.com/data/t81-558/sentences.csv",na_values=['?'])


template = ChatPromptTemplate.from_template("Extract the person name from this sentence: '{sentence}'.")


inputs = [{'sentence': s} for s in df['sentence']]


chain = LLMChain(llm=llm, prompt=template)


result = [chain.run(input_data) for input_data in inputs]

df['name'] = result

df['name'] = df['name'].str.replace(r'\n+', '', regex=True)

df_submit = df[['id','name']]

# df_submit

# Submit
submit(source_file=file,data=[df_submit],key=key,course='t81-558',no=6)

Success: Submitted Assignment 6 (t81-558) for l.zhijiang:
You have submitted this assignment 3 times. (this is fine)
No errors, warnings, or notes on your data. Rock on! You will probably do well, but no guarantee. :-)
